In [1]:
import os 
## Because the KERAS used is a old verion, KERAS 1
os.environ['TF_USE_LEGACY_KERAS'] = '1'

## Allows memory allocation on the GPU to be done asynchronously, 
## potentially improving performance and efficiency in certain scenarios.
os.environ['TF_GPU_ALLOCATOR'] = 'cuda_malloc_async'

## Specify which GPUs are available to a program.
## The list of GPUs is usually represented by indices starting at 0.
#os.environ['CUDA_VISIBLE_DEVICES'] = '-1' # disable cuda

# Install VideoImageTools

To install VideoImageTools go to next link https://github.com/trucomanx/VideoImageTools

# Install OpenPifPafTools

To install OpenPifPafTools go to next link https://github.com/trucomanx/OpenPifPafTools


In [2]:
input_default_json_conf_file='testing_over_video.json';

# Bibliotecas externas

In [3]:
import platform, sys, os

import hashlib
import json
import VideoImageTools as vit
import OpenPifPafTools.OpenPifPafAnnotations as opp
import OpenPifPafTools.OpenPifPafGetData as oppgd

import cv2
import openpifpaf
from PIL import Image

/home/fernando/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


In [4]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

2024-09-04 21:08:48.193897: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-04 21:08:48.204682: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-04 21:08:48.208054: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-04 21:08:48.216057: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-04 21:08:48.774276: W tensorflow/compiler/tf2

Num GPUs Available:  1


I0000 00:00:1725494929.188886 1364278 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1725494929.211547 1364278 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1725494929.211651 1364278 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355


# Biblioteca local

In [5]:
sys.path.append('../library');

In [6]:
import SystemEmotion4Lib.Classifier as mylib

# Variables

In [7]:
## Load json conf file
fd = open(os.path.join('./',input_default_json_conf_file));
DATA = json.load(fd);
fd.close()

In [8]:
## Model of network
dataset_name         = DATA['dataset_name']; # nome do dataset usado no treino

checkpoint = DATA["checkpoint"];
model_type_body = DATA["model_type_body"]; ## 'mobilenet_v3', 'efficientnet_b3', 'inception_v3', 'inception_resnet_v2', 'resnet_v2_50'
model_type_face = DATA["model_type_face"]; ## 'mobilenet_v3', 'efficientnet_b3', 'inception_v3', 'inception_resnet_v2', 'resnet_v2_50'
model_type_skel = DATA["model_type_skel"];
model_type_fusion = DATA["model_type_fusion"];
enable_minus = DATA["enable_minus"];

vin_path = DATA['input_mp4_file'];

output_base_dir = DATA['output_base_dir'];

sub_dir='sub_dir1';

body_factor=DATA['body_factor'];
face_factor=DATA['face_factor'];
face_detector_method=DATA['face_detector_method'];

dimage_method=True;


# If command line

In [9]:
for n in range(len(sys.argv)):
    if   sys.argv[n]=='--model-type-check':
        checkpoint=sys.argv[n+1];
    elif sys.argv[n]=='--model-type-body':
        model_type_body=sys.argv[n+1];
    elif sys.argv[n]=='--model-type-face':
        model_type_face=sys.argv[n+1];
    elif sys.argv[n]=='--model-type-skel':
        model_type_skel=int(sys.argv[n+1]);
    elif sys.argv[n]=='--model-type-fusion':
        model_type_fusion=int(sys.argv[n+1]);
    elif sys.argv[n]=='--enable-minus':
        enable_minus=sys.argv[n+1].lower()=='true';
    elif sys.argv[n]=='--input-file':
        vin_path=sys.argv[n+1];
    elif sys.argv[n]=='--output-dir':
        output_base_dir=sys.argv[n+1];
    elif sys.argv[n]=='--sub-dir':
        sub_dir=sys.argv[n+1];


INFO=dict();
INFO["model_type_body"] = model_type_body; 
INFO["model_type_face"] = model_type_face;
INFO["model_type_skel"] = model_type_skel;
INFO["model_type_fusion"] = model_type_fusion;
INFO["enable_minus"] = enable_minus;
INFO['body_factor'] = body_factor;
INFO['face_factor'] = face_factor;
INFO['face_detector_method'] = face_detector_method;

# Serializar o dicionário em uma string JSON com uma ordenação consistente
json_string = json.dumps(INFO, sort_keys=True)
# Gerar um hash MD5 a partir da string JSON
hash_object = hashlib.md5(json_string.encode())
sub_dir = hash_object.hexdigest()



print('')
print('       checkpoint:',checkpoint);
print('  model_type_body:',model_type_body);
print('  model_type_face:',model_type_face);
print('  model_type_skel:',model_type_skel);
print('model_type_fusion:',model_type_fusion);
print('     enable_minus:',enable_minus);
print('         vin_path:',vin_path);
print('  output_base_dir:',output_base_dir);
print('          sub_dir:',sub_dir);
print('      body_factor:',body_factor);
print('      face_factor:',face_factor);
print('')



       checkpoint: shufflenetv2k16
  model_type_body: efficientnet_b3
  model_type_face: efficientnet_b3
  model_type_skel: 20
model_type_fusion: 11
     enable_minus: False
         vin_path: /mnt/8811f502-ae19-4dd8-8371-f1915178f581/Fernando/DATASET/TESE/PATIENT-RECOGNITION/PATIENT-VIDEOS/dataset-toy/drhouse_mini_cut.mp4
  output_base_dir: /mnt/8811f502-ae19-4dd8-8371-f1915178f581/Fernando/OUTPUTS/DOCTORADO2/system_emotion4_1
          sub_dir: 70b394495c04560405a4b2e32faed4de
      body_factor: 0.95
      face_factor: 0.95



In [10]:
vout_dir_path=os.path.join(output_base_dir,dataset_name,sub_dir,'test_over_video');
print('vout_dir_path:',vout_dir_path);

vout_dir_path: /mnt/8811f502-ae19-4dd8-8371-f1915178f581/Fernando/OUTPUTS/DOCTORADO2/system_emotion4_1/ber2024-source/test_over_video/70b394495c04560405a4b2e32faed4de


In [11]:

vout_path=os.path.join(vout_dir_path,os.path.basename(vin_path));
print('vout_path:',vout_path);

vout_path: /mnt/8811f502-ae19-4dd8-8371-f1915178f581/Fernando/OUTPUTS/DOCTORADO2/system_emotion4_1/ber2024-source/test_over_video/70b394495c04560405a4b2e32faed4de/drhouse_mini_cut.mp4


# Creating output directory

In [12]:
os.makedirs(vout_dir_path,exist_ok=True); 
print('Created output_dir:',vout_dir_path);

Created output_dir: /mnt/8811f502-ae19-4dd8-8371-f1915178f581/Fernando/OUTPUTS/DOCTORADO2/system_emotion4_1/ber2024-source/test_over_video/70b394495c04560405a4b2e32faed4de


In [13]:


# Salvando o dicionário como JSON
with open(os.path.join(vout_dir_path,os.path.basename(vin_path)+".json"), "w") as file:
    json.dump(INFO, file, indent=4)
    

# Auxiliar functions

In [14]:
import numpy as np
import tensorflow as tf
import torch

def extra_func(cap):
    # Libera a memória da GPU no PyTorch
    torch.cuda.empty_cache()
    
    # Limpa a sessão atual e libera a memória da GPU no TensorFlow
    tf.keras.backend.clear_session()
    print('clean')

def my_func(Clf,frame):
    #categories=['angry','disgusted','fearful','happy','neutral','sad','surprised'];
    categories=['negative','neutral','pain','positive'];
    
    img_tmp = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB);
    pil_im = Image.fromarray(img_tmp);
    
    res, res_face, res_body, res_skel, face_bbox, body_bbox = Clf.predict_all_pil(pil_im);

    if res is None:
        return frame;
    
    res = np.argmax(res);
    texto=categories[res];
    
    (xi,yi,xo,yo) = body_bbox;
    
    thickness=2;
    
    frame = cv2.putText(frame,
                        texto,
                        org = (int(xi), int((yi+yo)/2)),
                        fontFace = cv2.FONT_HERSHEY_DUPLEX,
                        fontScale = 2.0,
                        color = (255, 0, 0),
                        thickness = thickness
                        );

    color=(0,255,0);
    cv2.rectangle(frame,(xi,yi),(xo,yo),color,thickness);

    if isinstance(face_bbox, tuple):
        (xi,yi,xo,yo) = face_bbox;
        color=(0,0,255);
        cv2.rectangle(frame,(xi,yi),(xo,yo),color,thickness);
    
    return frame;




In [15]:
Clf=mylib.Emotion4Classifier(   checkpoint=checkpoint,
                                model_type_face=model_type_face,
                                model_type_body=model_type_body,
                                model_type_skel=model_type_skel,
                                model_type_skel_enable_minus=enable_minus,
                                model_type_fusion=model_type_fusion,
                                body_factor=body_factor,
                                face_factor=face_factor,
                                face_detector_method=face_detector_method);

if dimage_method==True:
    ## Crio pasata com imagens
    vout_dir_tmp=os.path.join(vout_dir_path,os.path.splitext(os.path.basename(vin_path))[0]);
    image_files, fps = vit.apply_func_predictor_over_video_to_frames(my_func,Clf,vin_path,vout_dir_tmp, show=False,FORMATO = "frame_{:05d}.png");
    
    ## Save frames to video
    vit.images_to_video(image_files, fps, vout_path)
else:
    vit.apply_func_predictor_over_video_outmp4(my_func,Clf,vin_path,vout_path,extra_func_counter=1024,extra_func=extra_func)

print('working end')

I0000 00:00:1725494929.598270 1364278 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1725494929.598371 1364278 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1725494929.598437 1364278 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1725494929.654327 1364278 cuda_executor.cc:1015] successful NUMA node read from SysFS ha


Transfer learning architecture efficientnet_b3
        url: https://tfhub.dev/tensorflow/efficientnet/b3/feature-vector/1
target_size: (300, 300)
Loading the weights in: /home/fernando/.local/lib/python3.10/site-packages/FaceEmotion4Lib/models/model_efficientnet_b3.h5
Loaded the weights in: /home/fernando/.local/lib/python3.10/site-packages/FaceEmotion4Lib/models/model_efficientnet_b3.h5

Transfer learning architecture efficientnet_b3
        url: https://tfhub.dev/tensorflow/efficientnet/b3/feature-vector/1
target_size: (300, 300)
Loading the weights in: /home/fernando/.local/lib/python3.10/site-packages/BodyEmotion4Lib/models/model_efficientnet_b3.h5
Loaded the weights in: /home/fernando/.local/lib/python3.10/site-packages/BodyEmotion4Lib/models/model_efficientnet_b3.h5

Loading the weights in: /home/fernando/.local/lib/python3.10/site-packages/SkeletonEmotion4Lib/models/model_onlycls_ncod20.h5
Loaded the weights in: /home/fernando/.local/lib/python3.10/site-packages/SkeletonEmotion

Processing:   0%|                                         [ time left: ? ]

src/openpifpaf/csrc/src/cif_hr.cpp:102: UserInfo: resizing cifhr buffer
src/openpifpaf/csrc/src/occupancy.cpp:53: UserInfo: resizing occupancy buffer


2024-09-04 21:09:04.677121: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 8907
W0000 00:00:1725494944.688381 1364378 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1725494944.713117 1364378 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1725494944.713891 1364378 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1725494944.714683 1364378 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1725494944.715463 1364378 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1725494944.720931 1364378 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1725494944.722411 1364378 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1725494944.723734 1364378 gpu_t

Vídeo criado com sucesso em: /mnt/8811f502-ae19-4dd8-8371-f1915178f581/Fernando/OUTPUTS/DOCTORADO2/system_emotion4_1/ber2024-source/test_over_video/70b394495c04560405a4b2e32faed4de/drhouse_mini_cut.mp4
working end
